# Nhận diện khuôn mặt bằng OpenCV DNN

#### Haar Cascades
#### CNN - Convolutional Neural Networks
##### Cần tìm hiểu về ResNet-10, ResNet

In [2]:
from library import *

### Xử lý trong 01 ảnh/Anhr Tĩnh

In [2]:
# Tải ảnh đầu vào
img = cv2.imread('./data/face.png')

# HIển thị ảnh gốc
cv2.imshow('Original', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:

# Tải mô hình đã được huấn luyện từ trước
net = cv2.dnn.readNetFromCaffe('./model/deploy.prototxt',
                               './model/res10_300x300_ssd_iter_140000_fp16.caffemodel')

# Đọc ảnh đầu vào
img = cv2.imread('./data/face.png')
h, w = img.shape[:2]

# Nếu cho nhiều hình ảnh thì cho đoạn phía dưới cho vào vòng for

# img: Đây là ảnh đầu vào mà bạn muốn nhận dạng khuôn mặt. Trong trường hợp của bạn, bạn đã đọc ảnh từ tệp "face.png".
# 1.0: Đây là tỷ lệ co giãn cho ảnh. Trong trường hợp này, ảnh sẽ không bị co giãn hoặc mở rộng, và giữ nguyên kích thước ban đầu.
# (300, 300): Đây là kích thước mà mô hình yêu cầu cho ảnh đầu vào. Mô hình mà bạn đang sử dụng mong muốn ảnh có kích thước 300x300 pixel. Do đó, bạn co giãn hoặc cắt ảnh đầu vào thành kích thước này.
# (104, 177, 123): Đây là giá trị trung bình màu sắc được trừ đi từ mỗi pixel của ảnh. Điều này thường được sử dụng để chuẩn hóa dữ liệu đầu vào. Trong trường hợp này, các giá trị này thường được lấy từ dữ liệu huấn luyện của mô hình.
# swapRB=False: Đây là một cờ để xác định xem có cần hoán đổi các kênh màu đỏ và xanh (Red-Blue) trong ảnh hay không. Trong trường hợp này, bạn đã đặt nó thành False, tức là không hoán đổi kênh mà
# Chuẩn hóa ảnh đầu vào theo yêu cầu mô hình
blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), (104, 177, 123), swapRB=False)
net.setInput(blob)

# Chạy mạng để phát hiện khuôn mặt
faces = net.forward()

# Danh sách để lưu kết quả phát hiện khuôn mặt
confidences = []
boxes = []

# Duyệt từng khuôn mặt đã được phát hiện
for i in range(0, faces.shape[2]):
    # 0st:  batch index, vì chỉ có 1 ảnh, nên batch=0
    # 0nd: chỉ số lớp đầu ra, mô hình chỉ có 1 lớp đầu ra nên chỉ số lớp cũng là 0
    # i: chỉ khuôn mặt thứ i
    # : lấy tất cả 7 thông tin của khuôn mặt
    confidence = faces[0, 0, i, 2]
    if confidence > 0.5:  # Chỉ lấy các khuôn mặt có độ tin cậy > 0.5
        startx = max(0, int(faces[0, 0, i, 3] * w))
        starty = max(0, int(faces[0, 0, i, 4] * h))
        endx = min(w, int(faces[0, 0, i, 5] * w))
        endy = min(h, int(faces[0, 0, i, 6] * h))
        
        boxes.append([startx, starty, endx - startx, endy - starty])  # X, Y, Width, Height
        confidences.append(confidence)

# In thông tin trước khi thực hiện NMS

print("Các hộp trước NMS:")
for box, confidence in zip(boxes, confidences):
    print(f"Hộp: {box}, Độ tin cậy: {confidence:.2f}")

# Thực hiện Non-Maximum Suppression (NMS)
# Mục tiêu của NMS:
# Giữ lại khung chứa tốt nhất (độ tin cậy cao nhất).
# Loại bỏ các khung chứa dư thừa, trùng lặp khác.
indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.3)

# Vẽ các hộp đã lọc qua NMS
print("\nCác hộp sau NMS:")
for i in indices.flatten():  # Flatten đảm bảo chỉ số được duyệt qua chính xác
    box = boxes[i]
    startx, starty, width, height = box
    endx = startx + width
    endy = starty + height
    
    print(f"Hộp: {box}, Độ tin cậy: {confidences[i]:.2f}")
    
    # Vẽ khung và hiển thị độ tin cậy
    cv2.rectangle(img, (startx, starty), (endx, endy), (0, 255, 255), 2)
    text = 'Face: {:.2f}%'.format(confidences[i] * 100)
    cv2.putText(img, text, (startx, starty - 10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255))

# Hiển thị kết quả
cv2.imshow('Kết quả', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


### XỬ LÝ CHO VIDEO/WEBCAM

In [ ]:
# Kiểm tra lại backend và target sau khi cấu hình
print("Backend:", net.getPreferableBackend())
print("Target:", net.getPreferableTarget())

In [ ]:
import cv2

# Tải mô hình đã được huấn luyện từ trước
net = cv2.dnn.readNetFromCaffe('./model/deploy.prototxt', './model/res10_300x300_ssd_iter_140000_fp16.caffemodel')

# Mở Webcam
cap = cv2.VideoCapture(0)
# cv2.namedWindow('Camera Detect', cv2.WINDOW_NORMAL)

while True:
    # Đọc khung hình từ camera
    ret, frame = cap.read()
    if not ret:
        break

    # Lấy kích thước khung hình
    h, w = frame.shape[:2]

    # Chuẩn hóa ảnh đầu vào theo yêu cầu mô hình
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104, 177, 123), swapRB=False)
    net.setInput(blob)

    # Chạy mô hình để phát hiện khuôn mặt
    faces = net.forward()

    # Danh sách để lưu kết quả phát hiện khuôn mặt
    confidences = []
    boxes = []

    # Duyệt qua các khuôn mặt được phát hiện
    for i in range(faces.shape[2]):
        confidence = faces[0, 0, i, 2]
        if confidence > 0.1:  # Chỉ lấy khuôn mặt có độ tin cậy > 0.5
            startx = max(0, int(faces[0, 0, i, 3] * w))
            starty = max(0, int(faces[0, 0, i, 4] * h))
            endx = min(w, int(faces[0, 0, i, 5] * w))
            endy = min(h, int(faces[0, 0, i, 6] * h))

            boxes.append([startx, starty, endx - startx, endy - starty])
            confidences.append(confidence)

    # Thực hiện Non-Maximum Suppression (NMS)
    indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.7)

    # Vẽ các khung hình sau khi lọc NMS
    for i in indices.flatten():
        box = boxes[i]
        startx, starty, width, height = box
        endx, endy = startx + width, starty + height

        # Vẽ khung và hiển thị độ tin cậy
        cv2.rectangle(frame, (startx, starty), (endx, endy), (0, 255, 255), 2)
        text = 'Face: {:.2f}%'.format(confidences[i] * 100)
        cv2.putText(frame, text, (startx, starty - 10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255))

    # Hiển thị kết quả
    cv2.imshow('Camera Detect', frame)
    if cv2.waitKey(1) == ord('q'):
        break

# Giải phóng camera và đóng cửa sổ
cap.release()
cv2.destroyAllWindows()
